# Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khorinsk,RU,52.1657,109.7726,-0.11,99,100,2.84,overcast clouds
1,1,Eyl,SO,7.9803,49.8164,75.09,71,100,16.40,overcast clouds
2,2,Kruisfontein,ZA,-34.0033,24.7314,75.88,58,100,14.34,overcast clouds
3,3,Bredasdorp,ZA,-34.5322,20.0403,66.27,66,0,3.91,clear sky
4,4,Igrim,RU,63.1906,64.4162,-10.01,100,72,4.18,broken clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Eyl,SO,7.9803,49.8164,75.09,71,100,16.40,overcast clouds
2,2,Kruisfontein,ZA,-34.0033,24.7314,75.88,58,100,14.34,overcast clouds
11,11,Itarema,BR,-2.9248,-39.9167,84.34,65,100,13.89,overcast clouds
13,13,Mapiripan,CO,2.8892,-72.1256,89.65,42,100,6.31,overcast clouds
14,14,Cidreira,BR,-30.1811,-50.2056,81.25,51,0,11.86,clear sky


In [28]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

195


City_ID                195
City                   195
Country                194
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

194


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Eyl,SO,7.9803,49.8164,75.09,71,100,16.40,overcast clouds
2,2,Kruisfontein,ZA,-34.0033,24.7314,75.88,58,100,14.34,overcast clouds
11,11,Itarema,BR,-2.9248,-39.9167,84.34,65,100,13.89,overcast clouds
13,13,Mapiripan,CO,2.8892,-72.1256,89.65,42,100,6.31,overcast clouds
14,14,Cidreira,BR,-30.1811,-50.2056,81.25,51,0,11.86,clear sky


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Eyl,SO,75.09,overcast clouds,7.9803,49.8164,
2,Kruisfontein,ZA,75.88,overcast clouds,-34.0033,24.7314,
11,Itarema,BR,84.34,overcast clouds,-2.9248,-39.9167,
13,Mapiripan,CO,89.65,overcast clouds,2.8892,-72.1256,
14,Cidreira,BR,81.25,clear sky,-30.1811,-50.2056,
19,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,
20,Rikitea,PF,79.09,clear sky,-23.1203,-134.9692,
21,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,
22,San Cristobal,VE,79.99,broken clouds,7.7669,-72.2250,
33,Batticaloa,LK,76.86,overcast clouds,7.7102,81.6924,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
 
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Eyl,SO,75.09,overcast clouds,7.9803,49.8164,Amiin Hotel
2,Kruisfontein,ZA,75.88,overcast clouds,-34.0033,24.7314,Oyster Bay House Rental
11,Itarema,BR,84.34,overcast clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
13,Mapiripan,CO,89.65,overcast clouds,2.8892,-72.1256,Monserrate
14,Cidreira,BR,81.25,clear sky,-30.1811,-50.2056,Hotel Castelo
19,Arraial Do Cabo,BR,87.76,clear sky,-22.9661,-42.0278,Pousada Porto Praia
20,Rikitea,PF,79.09,clear sky,-23.1203,-134.9692,People ThankYou
21,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,Paradise Inn
22,San Cristobal,VE,79.99,broken clouds,7.7669,-72.2250,Posada Villaven C.A.
33,Batticaloa,LK,76.86,overcast clouds,7.7102,81.6924,Grand Star Guest House


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))